### Initial Raw Tests

In [1]:
# pip install mysql-connector-python
import mysql.connector 
import pandas as pd

In [2]:
# Connect to a local MySQL DB. in a real case scenario, this would probably be done via API to a cloud SQL instance.
conn = mysql.connector.connect(
    host="localhost",
    user="lucas",
    password="foo123",
    database="MetLifeChallenge"
)

In [3]:
cursor = conn.cursor()

In [55]:
# Create the 'training_dataset' table
create_table_query = """
CREATE TABLE IF NOT EXISTS training_dataset (
    id INT AUTO_INCREMENT PRIMARY KEY,
    age INT, 
    sex VARCHAR(255), 
    bmi DOUBLE, 
    children INT, 
    smoker VARCHAR(255),
    region VARCHAR(255),
    charges DOUBLE
)
"""
cursor.execute(create_table_query)

Index(['age', 'sex', 'bmi', 'children', 'smoker', 'region', 'charges'], dtype='object')

In [50]:
# Load CSV data and Insert into the table
data = pd.read_csv('Dataset/dataset.csv')

for _, row in data.iterrows():
    insert_query = "INSERT INTO training_dataset (age, sex, bmi, children, smoker, region, charges) VALUES (%s, %s, %s, %s, %s, %s, %s)"
    cursor.execute(insert_query, tuple(row))

In [51]:
conn.commit()
#conn.close()

In [57]:
# testing purposes
cursor = conn.cursor()

select_query = '''
SELECT * FROM training_dataset
'''
cursor.execute(select_query)
results = cursor.fetchall()

In [59]:
print(results[10])

(11, 25, 'male', 26.22, 0, 'no', 'northeast', 2721.3208)


In [ ]:
conn.close()

### Testing MYSQL implementation

In [14]:
from Utils.db_builder import MySQLBuilder

In [15]:
db_name = "MetLifeChallenge"
user = "lucas"
password = "foo123"
host = "localhost"
table_name = "training_dataset"
df_path = "Dataset/dataset.csv"

builder = MySQLBuilder(
    db_name = db_name, 
    table_name = table_name, 
    df_path = df_path, 
    host = host, 
    user = user,
    pwd = password
)



 > MySQL helper initialized. Building Database instance.


In [17]:
# create table and load data
builder.create_training_table()
#builder.load_csv_data()

In [9]:
# delete data
delete_query = '''DELETE FROM training_dataset'''
builder.execute_sql_query(delete_query)

[]

In [18]:
builder.print_rows()

(2677, 19, 'female', 27.9, 0, 'yes', 'southwest', 16884.924)
(2678, 18, 'male', 33.77, 1, 'no', 'southeast', 1725.5523)
(2679, 28, 'male', 33.0, 3, 'no', 'southeast', 4449.462)
(2680, 33, 'male', 22.705, 0, 'no', 'northwest', 21984.47061)
(2681, 32, 'male', 28.88, 0, 'no', 'northwest', 3866.8552)
(2682, 31, 'female', 25.74, 0, 'no', 'southeast', 3756.6216)
(2683, 46, 'female', 33.44, 1, 'no', 'southeast', 8240.5896)
(2684, 37, 'female', 27.74, 3, 'no', 'northwest', 7281.5056)
(2685, 37, 'male', 29.83, 2, 'no', 'northeast', 6406.4107)
(2686, 60, 'female', 25.84, 0, 'no', 'northwest', 28923.13692)


### Testing training pipeline

In [1]:
from Pipelines.training import Trainer

In [2]:
db_name = "MetLifeChallenge"
user = "lucas"
pwd = "foo123"
host = "localhost"
table_name = "training_dataset"
df_path = "Dataset/dataset.csv"

trainer = Trainer(db_name, user, pwd, host, table_name, df_path)


 > Initialized Training Pipeline.


In [3]:
trainer.orchestrator()


 > Starting Training process.

 > Extracting data from MySQL database.

 > MySQL helper initialized. Building Database instance.

 > Showing first 5 rows: 
       age     sex     bmi  children smoker     region      charges
col1                                                              
2677   19  female  27.900         0    yes  southwest  16884.92400
2678   18    male  33.770         1     no  southeast   1725.55230
2679   28    male  33.000         3     no  southeast   4449.46200
2680   33    male  22.705         0     no  northwest  21984.47061
2681   32    male  28.880         0     no  northwest   3866.85520

 > Preprocessing data. Encoding categorical values with OneHotEncoding, deleting nulls if any, replacing missing values. etc.

 > Performed an 80/20 split. Training set has 1070 examples and 16 predictor variables. Test set has 268 examples.

 > Scaling numerical variables. Make sure this process is always performed after splitting your data to avoid data leakage.

 > T

### Testing Scoring Pipeline

In [1]:
from Pipelines.scoring import Scorer

In [2]:
db_name = "MetLifeChallenge"
user = "lucas"
pwd = "foo123"
host = "localhost"
table_name = "training_dataset"
df_path = "Dataset/dataset.csv"

scorer = Scorer(db_name, user, pwd, host, table_name, df_path)


 > Initialized Scoring Pipeline.


In [3]:
scorer.orchestrator()


 > Initialized Scoring process.

 > Creating random samples.

 > MySQL helper initialized. Building Database instance.

 > Loading BestModel from memory.

 > Predicting charges...

 > Creating Scoring table on MySQL.

 ----> 🧠 Reported performance on sample data for RMSE: 3978.8979017958227 🧠 <----

 First records from scoring table: 
 
      age     sex     bmi  children smoker     region      charges  \
col1                                                                 
1      37    male  34.100         4    yes  southwest  40182.24600   
2      25    male  24.130         0    yes  northwest  15817.98570   
3      54    male  30.210         0     no  northwest  10231.49990   
4      53  female  36.860         3    yes  northwest  46661.44240   
5      41  female  33.155         3     no  northeast   8538.28845   

      predicted_charges  
col1                     
1          39695.532928  
2          18195.091772  
3          12478.290581  
4          45165.585439  
5           8